In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of skaro, before pip installed version, for debugging
import pathlib
import sys
import os

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yt

from skaro.analysis import count_planets
from skaro.model import Model
from skaro.utilities.dataframe import rename_labels
from skaro.utilities.math import calculate_pca
from skaro.visualization.visualization_utils import (
    FigureProcessor,
    adjust_legend,
    get_palette,
    ridgeplot,
    set_plot_defaults,
)

## DATA SETUP

In [4]:
snapshot = 127
resolution = 8192
sim_id = "37_11"
num_embryos = 50
host_star_masses = 1
planet_categories = [
    "Earth",
    "Super-Earth",
    "Neptunian",
    "Sub-Giant",
    "Giant",
]

In [5]:
model = Model(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)
ds = model.ds
mw = model.halo

yt : [INFO     ] 2023-11-25 17:40:56,664 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2023-11-25 17:40:56,818 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2023-11-25 17:40:56,819 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-11-25 17:40:56,820 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-25 17:40:56,821 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2023-11-25 17:40:56,822 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-11-25 17:40:56,823 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2023-11-25 17:40:56,824 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2023-11-25 17:40:56,825 Parameters: omega_matter              = 0.318
yt : [INFO     ] 2023-11-25 17:40:56,825 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-11-25 17:40:56,826 Parameters: hubble_constant      

## PLOT SETUP

In [6]:
set_plot_defaults()

figure_subdirectory = f"MilkyWayAnalogue/{model.resolution}/{model.sim_id}/{model.snapshot}/Planet_numbers"
save = True

## CREATE DATA SOURCE

In [7]:
radius = 0.2 * mw.virial_radius()

## ABSOLUTE NUMBER PER PLANET TYPE

In [8]:
def aggregate_planet_counts_for_embryos(host_star_mass, embryo_list):
    counts_by_embryos = {
        num_embryos: count_planets(
            model=model,
            data_creator=lambda: mw.sphere(radius=radius),
            planet_categories=planet_categories,
            model_config={
                "ngpps_star_masses": host_star_mass,
                "ngpps_num_embryos": num_embryos,
            },
            components="stars",
        )
        for num_embryos in embryo_list
    }
    counts_dataframe = pd.concat(counts_by_embryos).reset_index(level=1, drop=True)
    counts_dataframe["Total"] = counts_dataframe.sum(axis=1)
    return counts_dataframe.rename_axis(r"$N_\mathrm{Embryos}$")

In [9]:
embryo_list = (10, 20, 50, 100)

planet_counts = aggregate_planet_counts_for_embryos(
    host_star_mass=host_star_masses, embryo_list=embryo_list
)


      CONFIG: Setting ngpps_num_embryos to 10.


      TIMER: Block 'Adding Planets...' took 1.99 seconds.


      CONFIG: Setting ngpps_num_embryos to 20.


      TIMER: Block 'Adding Planets...' took 2.42 seconds.


      CONFIG: Setting ngpps_num_embryos to 50.


      TIMER: Block 'Adding Planets...' took 3.26 seconds.


      CONFIG: Setting ngpps_num_embryos to 100.


      TIMER: Block 'Adding Planets...' took 4.2 seconds.



In [10]:
planet_counts.div(1e9).round(decimals=1).T

$N_\mathrm{Embryos}$,10,20,50,100
Earth,3.0,5.2,9.4,11.0
Super-Earth,3.7,5.1,9.1,11.0
Neptunian,1.3,1.4,1.3,1.4
Sub-Giant,0.4,0.4,0.3,0.3
Giant,0.8,0.8,0.7,0.7
Total,9.2,12.9,20.8,24.5
